In [2]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import pandas as pd

import numpy as np
import os
import shutil
import posixpath
import wfdb
import seaborn as sn

In [3]:
# docs: https://wfdb.readthedocs.io/en/latest/wfdb.html
# SpO2が含まれているデータは a01r のようにrが付いている
# records

# 'a01r', 'a02r', 'a03r', 'a04r', 'b01r', 'c01r', 'c02r', 'c03r'
subjects = ['a01r', 'a02r', 'a03r', 'a04r', 'b01r', 'c01r', 'c02r', 'c03r']
records = {}

# データの読み込み
for subject in subjects:
    path = "dataset/" + subject
    records[subject] = wfdb.rdrecord(path)  # 計測データ
    records['ann_' + subject] = wfdb.rdann(path, 'apn', shift_samps=True) # ラベル


In [4]:
# https://archive.physionet.org/physiobank/database/apnea-ecg/annotations.shtml
# display(annotation.__dict__)
display(len(records['ann_a01r'].sample))
display(len(records['ann_a01r'].symbol))

3108000 / 6000
print(records['ann_a01r'].symbol)

489

489

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',

In [5]:
# 特徴量の抽出、学習用のデータ作成
# 1 sample(6000個のデータ)毎に基本統計量を出してそれをInputに与える
# 例) record.p_signal[0:6000], record.p_signal[6001:12001]で統計量を出す
        
def extract_feature(data):
    return np.hstack([data.mean().values, data.std().values, data.max().values, data.min().values, (data.max() - data.min()).values])

# train用
# 'b01r', 'c01r', 'c02r' はSpO2が0%のため除外、'c01r~c03rのデータはAのラベルが無い'
train_subjects = ['a01r', 'a02r', 'a04r', 'c03r']

# test用
test_subjects = ['a03r']

data_array = []
for subject in train_subjects:
    # annotation分だけ繰り返す
    ann_count = len(records['ann_'+subject].sample)
    data = pd.DataFrame(records[subject].p_signal, columns=records[subject].sig_name, dtype='float')
    data['SpO2'] = (data['SpO2'] / 100)
    data = data.fillna(method='ffill') # 直前の値を使って欠損値埋める

    start_idx = 0
    end_idx = 6000
    for n in range(0, ann_count):
        data_array.append(data[start_idx:end_idx].to_numpy())
        start_idx = end_idx + 1
        end_idx = end_idx + 6000
print(data_array[0])
        
test_data = []
for subject in test_subjects:
    ann_count = len(records['ann_'+subject].sample)
    data = pd.DataFrame(records[subject].p_signal, columns=records[subject].sig_name, dtype='float')
    data['SpO2'] = (data['SpO2'] / 100)
    data = data.fillna(method='ffill') # 直前の値を使って欠損値埋める
    start_idx = 0
    end_idx = 6000
    for n in range(0, ann_count):
        test_data.append(data[start_idx:end_idx].to_numpy())
        start_idx = end_idx + 1
        end_idx = end_idx + 6000

[[-0.1015  -0.3446   0.23545  0.98   ]
 [-0.1025  -0.3455   0.2364   0.98   ]
 [-0.1035  -0.3464   0.23735  0.98   ]
 ...
 [-0.0642   0.0325  -0.1116   0.98   ]
 [-0.0526   0.0328  -0.11325  0.98   ]
 [-0.04095  0.0332  -0.1149   0.98   ]]


In [6]:
# 学習データ作成
print(len(data_array))
train_X = data_array

print(train_X[0].size)
print(train_X[1])
print(type(train_X[1]))


#train_X.shape

# テストデータ作成
print(len(test_data))
test_X = test_data

# test_X.shape

1963
24000
[[-0.03055  0.03295 -0.11795  0.98   ]
 [-0.0318   0.03235 -0.11935  0.98   ]
 [-0.03305  0.03175 -0.12075  0.98   ]
 ...
 [-0.18845 -0.163    0.1757   0.97   ]
 [-0.1876  -0.1638   0.17755  0.97   ]
 [-0.18675 -0.1646   0.1795   0.97   ]]
<class 'numpy.ndarray'>
519


In [150]:
# NとAを1,0に変換する
from sklearn.preprocessing import LabelEncoder

label_data = []
for subject in train_subjects:
    le = LabelEncoder()
    le = le.fit(records['ann_' + subject].symbol)
    labels = le.transform(records['ann_' + subject].symbol)
    label_data = np.concatenate([label_data, labels], 0)

# テスト用のラベル
test_label = []
for subject in test_subjects:
    le = LabelEncoder()
    le = le.fit(records['ann_' + subject].symbol)
    test_label = le.transform(records['ann_' + subject].symbol)
    
display(label_data)
display(len(label_data))

display(test_label)
display(len(test_label))

array([1., 1., 1., ..., 0., 0., 0.])

1963

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

519

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from keras.utils import np_utils


# モデルの作成
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(6000, 4000)),
    keras.layers.Dense(100, activation='sigmoid'),
    keras.layers.Dense(100, activation='sigmoid'),
    keras.layers.Dense(1, activation='softmax')
])

In [ ]:
from keras import optimizers
from keras.utils import plot_model

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(train_X, label_data,
                    batch_size=128,   # 64,128みたいに2の倍数がよく使われるらしい
                    epochs=200)